In [1]:
import pandas as pd
import numpy as np

In [19]:
# Load data
patient_profile_df = pd.read_excel('synthetic_data v3.xlsx', sheet_name='patient_profile')
medication_orders_df = pd.read_excel('synthetic_data v3.xlsx', sheet_name='medication_orders')
reference_df = pd.read_excel('synthetic_data v3.xlsx', sheet_name='Reference')

In [12]:
patient_profile_df.head(3)

,patient_id,age,sex,height_m,weight_kg,hr_latest,sbp_latest,allergy,pmhx,issue,...,albumin,ck,fasting_glucose,hba1c,hdl,ldl,tg,microb,imaging,concomitant meds
0,1,71,F,1.56,51,92,113,NaN,"Parkinson disease, Type 2 Diabetes Mellitus, S...","Vertigo, Gastroesophageal reflux disease",...,NaN,NaN,7.1,6.5,0.5,2.0,2.0,NaN,Chest X-ray: No significant lung consolidation...,Oral Glipizide 15mg two times a day before mea...
1,2,70,M,1.62,81,83,109,NaN,"Child-Turcotte-Pugh Class C, Decompensated Cir...",Fall,...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CT Thorax Abdomen and Pelvis:\nExtensive right...,"Oral Bisoprolol 5mg every morning, Oral Aspiri..."
2,3,31,F,1.59,60,95,114,NaN,"Term Pregnancy, Rheumatoid arthritis","Delivery by emergency cesarean, currently brea...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Oral Lactulose 10mL TDS PRN for constipation, ..."


In [13]:
medication_orders_df.head(3)

,order_id,patient_id,route,medication,formulation,dose,dose_unit,freq,label,reason_fyi,open_fda_api,med_order_string
0,1.0,1.0,oral,METOCLOPRAMIDE,tablet,10.0,mg,three times a day,unsafe,Symptoms of Parkinson disease may be exacerbat...,https://api.fda.gov/drug/label.json?search=ope...,oral metoclopramide 10mg three times a day
1,2.0,17.0,oral,METOCLOPRAMIDE,tablet,10.0,mg,three times a day,unsafe,Symptoms of Parkinson disease may be exacerbat...,https://api.fda.gov/drug/label.json?search=ope...,oral metoclopramide 10mg three times a day
2,3.0,11.0,oral,METOCLOPRAMIDE,tablet,10.0,mg,three times a day,safe,The medication order is safe because it aligns...,https://api.fda.gov/drug/label.json?search=ope...,oral metoclopramide 10mg three times a day


In [14]:
reference_df.head(3)

,columns,description,display_name,uom
0,patient_id,id of patient,NaN,NaN
1,age,age of patient,NaN,years old
2,sex,"sex of patient (Male ""M"" or Female ""F"")",NaN,NaN


In [15]:
patient_profile_df.head(3)

,patient_id,age,sex,height_m,weight_kg,hr_latest,sbp_latest,allergy,pmhx,issue,...,albumin,ck,fasting_glucose,hba1c,hdl,ldl,tg,microb,imaging,concomitant meds
0,1,71,F,1.56,51,92,113,NaN,"Parkinson disease, Type 2 Diabetes Mellitus, S...","Vertigo, Gastroesophageal reflux disease",...,NaN,NaN,7.1,6.5,0.5,2.0,2.0,NaN,Chest X-ray: No significant lung consolidation...,Oral Glipizide 15mg two times a day before mea...
1,2,70,M,1.62,81,83,109,NaN,"Child-Turcotte-Pugh Class C, Decompensated Cir...",Fall,...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CT Thorax Abdomen and Pelvis:\nExtensive right...,"Oral Bisoprolol 5mg every morning, Oral Aspiri..."
2,3,31,F,1.59,60,95,114,NaN,"Term Pregnancy, Rheumatoid arthritis","Delivery by emergency cesarean, currently brea...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Oral Lactulose 10mL TDS PRN for constipation, ..."


In [42]:
def generate_patient_medication_summary(
    order_id: int,
    medication_orders_df: pd.DataFrame,
    patient_profile_df: pd.DataFrame,
    reference_df: pd.DataFrame
) -> str:
    """
    Generates a string summary combining the patient's profile information
    and a specific medication order, following the format of the given examples.
    """
    patient_profile_df.replace(np.nan, '', inplace=True) 
    
    # 1) Identify the row in medication_orders for the given order_id
    med_order_row = medication_orders_df.loc[medication_orders_df['order_id'] == order_id]
    if med_order_row.empty:
        return f"No medication order found for order_id = {order_id}."
    
    # 2) Extract the patient_id from that order
    patient_id = med_order_row['patient_id'].values[0]
    
    # 3) Identify the matching patient profile
    patient_row = patient_profile_df.loc[patient_profile_df['patient_id'] == patient_id]
    if patient_row.empty:
        return f"No patient profile found for patient_id = {patient_id} (order_id = {order_id})."
    
    # Convert row -> dictionary for easy access
    # patient_data_2 = patient_row    
    patient_data = patient_row.to_dict('records')[0]

    # 4) Basic demographics
    age = patient_data.get('age', 'N/A')
    sex = patient_data.get('sex', 'N/A')
    height = patient_data.get('height_m', 'null')  # e.g. in meters
    weight = patient_data.get('weight_kg', 'null')  # e.g. in kg
    heart_rate = patient_data.get('hr_latest', 'null')
    systolic_bp = patient_data.get('sbp_latest', 'null')

    # 5) Past medical history and admitting diagnoses
    #    (Assuming the data are stored in comma-separated strings, lists, etc.)
    past_medical_history = patient_data.get('pmhx', 'null')
    admitting_dx = patient_data.get('issue', 'null')

    # 6) Concomitant medications and drug allergy
    con_med = patient_data.get('concomitant meds', 'null')
    drug_allergy = patient_data.get('allergy', 'null')

    # 7) Microbiology report and imaging report
    micro_report = patient_data.get('microb', 'null')
    imaging_report = patient_data.get('imaging', 'null')

    # 8) Medication order from the medication_orders table
    #    "The doctor ordered oral metoclopramide 10mg three times a day."
    med_order_string = med_order_row['med_order_string'].values[0]

    # 9) Build the string for the top portion: demographics + PMHx + admitting dx
    #    Convert them into human-friendly strings.
    #    If these columns are stored as lists, you might do ", ".join(...).
    #    Below we assume they're stored as comma-separated text.
    pmhx_str = past_medical_history if past_medical_history else "null"
    admit_str = admitting_dx if admitting_dx else "null"
    con_med_str = con_med if con_med else "null"
    allergy_str = drug_allergy if drug_allergy else "null"

    top_section = (
        f"Patient is a {age} years old {'female' if sex == 'F' else 'male'}, height {height}m and weighs {weight}kg. "
        f"Latest heart rate is {heart_rate} bpm. "
        f"Latest systolic blood pressure is {systolic_bp}. "
        f"Patient has past medical history of {pmhx_str}. "
        f"Currently patient is admitted for {admit_str}. \n"
        f"Patient is also taking the following medications: {con_med_str}. \n"
        f"Patient has known allergy to: {allergy_str}.\n"
    )

    lab_lines = []
    for _, ref_row in reference_df.iterrows():
        col_name = ref_row['columns'] # e.g. "wbc"
        description = ref_row['description']
        display_name = ref_row['display_name'] # e.g. "White blood cell"
        uom = ref_row['uom']                   # e.g. "10^9/L"
    
        if col_name in patient_data and pd.notna(display_name):
            lab_value = patient_data[col_name]
            if lab_value != '':  # Only show if not null
                lab_lines.append(f"{display_name} = {lab_value} {uom}")
                
    # 11) Microbiology & imaging lines (always show "Microbiology Report" even if null)
    micro_line = f"Microbiology Report: {micro_report if micro_report else 'null'}"
    imaging_line = f"Imaging Report: {imaging_report if imaging_report else 'null'}"

    # Combine lab lines
    lab_section = ""
    if lab_lines:
        lab_section = (
            "Some recent lab results and reports are shown below:\n" +
            "\n".join(lab_lines) + "\n" +
            micro_line + "\n" +
            imaging_line + "\n"
        )
    else:
        # If no labs are present at all, we still might show Micro & Imaging lines
        lab_section = (
            "Some recent lab results and reports are shown below:\n" +
            micro_line + "\n" +
            imaging_line + "\n"
        )

    # 12) Medication order line
    med_order_line = f"The doctor ordered {med_order_string}."

    # 13) Final output
    final_text = (
        top_section + "\n" +
        lab_section + "\n" +
        med_order_line
    )

    return final_text.strip()

In [43]:
summary_1 = generate_patient_medication_summary(
        order_id=1,
        medication_orders_df=medication_orders_df,
        patient_profile_df=patient_profile_df,
        reference_df=reference_df
    )

In [44]:
print(summary_1)

Patient is a 71 years old female, height 1.56m and weighs 51kg. Latest heart rate is 92 bpm. Latest systolic blood pressure is 113. Patient has past medical history of Parkinson disease, Type 2 Diabetes Mellitus, Stable Ischemic Heart Disease. Currently patient is admitted for Vertigo,  Gastroesophageal reflux disease. 
Patient is also taking the following medications: Oral Glipizide 15mg two times a day before meal, Oral Clopidogrel 75mg every morning, Oral Benserazide 25 mg/levodopa 100 mg twice a day. 
Patient has known allergy to: null.

Some recent lab results and reports are shown below:
White Blood Cell = 2.32 10^9/L
Haemoglobin = 11.3 g/dL
Platelet = 56.0 g/dL
Creatinine Clearance = 61.0 mL/min
eGFR = 55.0 mL/min/1.73 m²
Sodium = 141 mmol/L
Potassium = 4.0 mmol/L
Magnesium = 0.8 mmol/L
Fasting Glucose = 7.1 mmol/L
HbA1C = 6.5 %
Hdl Hdl = 0.5 mmol/L
LDL = 2.0 mmol/L
TG = 2.0 mmol/L
Microbiology Report: null
Imaging Report: Chest X-ray: No significant lung consolidation, pleural 

In [49]:
context = []
j = 0
for i in range(50):
    summary = generate_patient_medication_summary(
        order_id=i+1,
        medication_orders_df=medication_orders_df,
        patient_profile_df=patient_profile_df,
        reference_df=reference_df
    )
    context.append(summary)

In [54]:
context[0]

'Patient is a 71 years old female, height 1.56m and weighs 51kg. Latest heart rate is 92 bpm. Latest systolic blood pressure is 113. Patient has past medical history of Parkinson disease, Type 2 Diabetes Mellitus, Stable Ischemic Heart Disease. Currently patient is admitted for Vertigo,  Gastroesophageal reflux disease. \nPatient is also taking the following medications: Oral Glipizide 15mg two times a day before meal, Oral Clopidogrel 75mg every morning, Oral Benserazide 25 mg/levodopa 100 mg twice a day. \nPatient has known allergy to: null.\n\nSome recent lab results and reports are shown below:\nWhite Blood Cell = 2.32 10^9/L\nHaemoglobin = 11.3 g/dL\nPlatelet = 56.0 g/dL\nCreatinine Clearance = 61.0 mL/min\neGFR = 55.0 mL/min/1.73 m²\nSodium = 141 mmol/L\nPotassium = 4.0 mmol/L\nMagnesium = 0.8 mmol/L\nFasting Glucose = 7.1 mmol/L\nHbA1C = 6.5 %\nHdl Hdl = 0.5 mmol/L\nLDL = 2.0 mmol/L\nTG = 2.0 mmol/L\nMicrobiology Report: null\nImaging Report: Chest X-ray: No significant lung con

In [60]:
df = pd.DataFrame({
    'order_id':range(1,51),
    'context':context    
    })

In [69]:
df.head()

,order_id,context
0,1,"Patient is a 71 years old female, height 1.56m..."
1,2,"Patient is a 65 years old male, height 1.8m an..."
2,3,"Patient is a 31 years old male, height 1.68m a..."
3,4,"Patient is a 30 years old female, height 1.64m..."
4,5,"Patient is a 70 years old male, height 1.62m a..."


In [73]:
df[df['order_id']==5]['context'].values[0]

'Patient is a 70 years old male, height 1.62m and weighs 81kg. Latest heart rate is 83 bpm. Latest systolic blood pressure is 109. Patient has past medical history of Child-Turcotte-Pugh Class C, Decompensated Cirrhosis, Ischemic Heart Disease, Acute Coronary Syndrome, Hyperlipidemia, Chronic Liver Disease. Currently patient is admitted for Fall. \nPatient is also taking the following medications: Oral Bisoprolol 5mg every morning, Oral Aspirin 100mg every morning. \nPatient has known allergy to: null.\n\nSome recent lab results and reports are shown below:\nWhite Blood Cell = 10.1 10^9/L\nHaemoglobin = 11.1 g/dL\nPlatelet = 125.0 g/dL\nUrea = 16.0 mmol/L\nCreatinine Clearance = 35.0 mL/min\nSodium = 132 mmol/L\nPotassium = 4.3 mmol/L\nMagnesium = 0.94 mmol/L\nCalcium = 1.99 mmol/L\nAST = 420.0 U/L\nALT = 90.0 U/L\nAlbumin = 20.0 g/L\nMicrobiology Report: null\nImaging Report: CT Thorax Abdomen and Pelvis:\nExtensive right hepatic lobe changes very suspicious for HCC. This is in the ar

In [74]:
# df.to_csv('context.csv', index=False)